## 8. Neural networks and deep learning

This week, we'll learn about neural nets and build a model for classifying images of clothes

### 8.1 Fashion classification
Dataset: <br>
* Full: https://github.com/alexeygrigorev/clothing-dataset <br>
* Small: https://github.com/alexeygrigorev/clothing-dataset-small

Links: <br>
* https://cs231n.github.io/

### 8.2 TensorFlow and Keras

* Installing TensorFlow
* Loading Images

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
path = '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/train/t-shirt'
name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
fullname = f'{path}/{name}'

load_img(fullname)

In [ ]:
img = load_img(fullname, target_size=(299,299))

In [ ]:
print(img)

In [ ]:
x = np.array(img)
x.shape

In [ ]:
x

### 8.3 Pre-trained Convolutional Neural Networks

* Imagenet dataset: https://www.image-net.org/
* Pre-trained models: https://keras.io/api/applications/

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [ ]:
model = Xception(
    weights='imagenet',
    input_shape = (299,299,3)
)

In [ ]:
X = np.array([x])

In [ ]:
X.shape

In [ ]:
X = preprocess_input(X)

In [ ]:
X[0]

In [ ]:
pred = model.predict(X)

In [ ]:
pred.shape

In [ ]:
decode_predictions(pred)

### 8.4 Convolutional neural networks

* Types of layers: convolutional and dense
* Convolutional layers and filters
* Dense layers

There are more layers. Read here: https://cs231n.github.io/

* Write a blog post about CNN

### 8.5 Transfer learning

* Reading data with <code>ImageDataGenerator</code>
* Train <code>Xception</code> on smaller images (150x150)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_ds = train_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/train', 
    target_size=(150,150), 
    batch_size=32
)

In [ ]:
X, y = next(train_ds)

In [ ]:
X.shape
# batch size, height, width, RGB

In [ ]:
train_ds.class_indices

In [ ]:
y[:5]

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/validation', 
    target_size=(150,150), 
    batch_size=32
)

In [ ]:
base_model = Xception(weights='imagenet',
                      include_top=False, # do not include dense layer
                      input_shape = (150,150,3)
                      )

base_model.trainable = False    # freeze convolutional layer

In [ ]:
inputs = keras.Input(shape=(150,150,3))

base = base_model(inputs, training=False)

# pooling = keras.layers.GlobalAveragePooling2D() #convert 3D image (base) to 1D --> Vector values
# vectors = pooling(base)

# simply using Functional Style
vectors = keras.layers.GlobalAveragePooling2D()(base)

# creating Dense Layers
outputs = keras.layers.Dense(10)(vectors)   # 10 - outputs, passing vectors as inputs

model = keras.Model(inputs, outputs)

In [ ]:
preds =  model.predict(X)

In [ ]:
preds.shape

In [ ]:
preds[0]

##### *Training the Model*

In [ ]:
# optimizers
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

# loss
loss = keras.losses.CategoricalCrossentropy(from_logits=True)


In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# epoch - How many times we train one batch, 10 epoch --> the training go over the data 10 times
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
# plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend();

### 8.6 Adjusting the learning rate

* What's the learning rate: (eg: How fast you read)
* Trying different values

In [ ]:
def make_model(learning_rate=0.01):
    base_model = Xception(weights='imagenet',
                      include_top=False, # do not include dense layer
                      input_shape = (150,150,3)
                      )

    base_model.trainable = False 

    ###########################################

    inputs = keras.Input(shape=(150,150,3))

    base = base_model(inputs, training=False)

    vectors = keras.layers.GlobalAveragePooling2D()(base)

    outputs = keras.layers.Dense(10)(vectors)

    model = keras.Model(inputs, outputs)

    ###########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [ ]:
scores = {}

for lr in [0.0001, 0.001, 0.01, 0.1]:
    print(lr)

    model = make_model(learning_rate=lr)
    # due to the time constraint, lets assume epoch --> 5
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history

    print()
    print()

In [ ]:
del scores[0.1]
del scores[0.0001]

In [ ]:
for lr, hist in scores.items():
    #plt.plot(hist['accuracy'], label=('train=%s' % lr))
    plt.plot(hist['val_accuracy'], label=('val=%s' % lr))

plt.xticks(np.arange(10))
plt.legend()
plt.show()

In [ ]:
learning_rate = 0.001

### 8.7 Checkpointing

* Saving the best model only
* Training a model with callbacks

In [ ]:
model.save('model_v1.h5', save_format='h5')

In [ ]:
'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5'.format(epoch=12, val_accuracy=0.844325)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001

model = make_model(learning_rate=learning_rate)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

### 8.8 Adding more layers

* Adding one inner dense layer
* Experimenting with different sizes of inner layer

In [ ]:
def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(weights='imagenet',
                      include_top=False, # do not include dense layer
                      input_shape = (150,150,3)
                      )

    base_model.trainable = False 

    ###########################################

    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    #Adding more layers, with activation ReLU
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)              
    
    outputs = keras.layers.Dense(10)(inner)

    model = keras.Model(inputs, outputs)

    ###########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [ ]:
learning_rate = 0.001

scores = {}

for size in [10, 100, 1000]:
    print(size)

    model = make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history

    print()
    print()

In [ ]:
# scores

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % size))

plt.xticks(np.arange(10))
plt.legend()

### 8.9 Regularization and Dropout

* Regularizing by freezing a part of the network
* Adding dropuout to our model
* Experimenting with different values

In [ ]:
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(weights='imagenet',
                      include_top=False, # do not include dense layer
                      input_shape = (150,150,3)
                      )

    base_model.trainable = False 

    ###########################################

    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    #Adding more layers, with activation ReLU
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)               

    # dropout
    drop = keras.layers.Dropout(droprate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)

    model = keras.Model(inputs, outputs)

    ###########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [ ]:
learning_rate = 0.001
size = 100

scores = {}

for droprate in [0.0, 0.2, 0.5, 0.8]:
    print(droprate)

    model = make_model(
        learning_rate=learning_rate, 
        size_inner=size,
        droprate=droprate
    )
    
    history = model.fit(train_ds, epochs=30, validation_data=val_ds)
    scores[droprate] = history.history

    print()
    print()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % droprate))

plt.ylim(0.78, 0.86)
plt.legend()

In [ ]:
hist = scores[0.0]
plt.plot(hist['val_accuracy'], label=0.0)

hist = scores[0.2]
plt.plot(hist['val_accuracy'], label=0.2)

plt.legend()
# plt.plot(hist['accuracy'], label=('val=%s' % droprate))

### 8.10 Data Augmentation

* Different data augmentations
* Training a model with augmentations
* How to select data augmentations?

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=1.0,
    zoom_range=0.1,
    vertical_flip=True
)

train_ds = train_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/train',
    target_size=(150,150),
    batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/validation',
    target_size=(150,150),
    batch_size=32,
    shuffle=False
)

In [ ]:
learning_rate = 0.001
size = 100
droprate = 0.2

model = make_model(
    learning_rate=learning_rate, 
    size_inner=size,
    droprate=droprate
)
    
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

In [ ]:
hist = history.history
plt.plot(hist['val_accuracy'], label='val')
plt.plot(hist['accuracy'], label='train')
plt.legend()

### 8.11 Training a Larger Model

* Train a 299x299 model

In [ ]:
def make_model(input_size = 150, learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(weights='imagenet',
                      include_top=False, # do not include dense layer
                      input_shape = (input_size,input_size,3)
                      )

    base_model.trainable = False 

    ###########################################

    inputs = keras.Input(shape=(input_size,input_size,3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    #Adding more layers, with activation ReLU
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)               

    # dropout
    drop = keras.layers.Dropout(droprate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)

    model = keras.Model(inputs, outputs)

    ###########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [ ]:
input_size = 299

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_ds = train_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/train',
    target_size=(input_size,input_size),
    batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/validation',
    target_size=(input_size,input_size),
    batch_size=32,
    shuffle=False
)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v4_{epoch:02d}_{val_accuracy:.3f}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.0005
size = 100
droprate = 0.2

model = make_model(
    input_size = 299,
    learning_rate=learning_rate, 
    size_inner=size,
    droprate=droprate
)
    
history = model.fit(train_ds, 
                    epochs=50, 
                    validation_data=val_ds,
                   callbacks=[checkpoint])

### 8.12 Using the model

* Loading the model
* Evaluating the model
* Getting predictions

In [7]:
import tensorflow as tf
from tensorflow import keras

import warnings

warnings.filterwarnings('ignore')

In [2]:
model = keras.models.load_model('/kaggle/working/xception_v4_06_0.883.keras')

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import preprocess_input

In [5]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = test_gen.flow_from_directory(
    '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/test',
    target_size=(299,299),
    batch_size=32,
    shuffle=False
)

Found 372 images belonging to 10 classes.


In [8]:
model.evaluate(test_ds)
#loss value & metric

12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 228ms/step - accuracy: 0.8979 - loss: 0.3051


[0.2525501847267151, 0.9139785170555115]

In [10]:
path = '/kaggle/input/clothing-dataset-small-master/clothing-dataset-small-master/test/pants/c8d21106-bbdb-4e8d-83e4-bf3d14e54c16.jpg'

In [16]:
img = load_img(path, target_size=(299,299))

In [17]:
import numpy as np

In [19]:
X = np.array(img)
X = np.array([X])
X.shape

(1, 299, 299, 3)

In [20]:
X = preprocess_input(X)

In [23]:
classes = ['dress','hat', 'longsleeve', 'outwear', 'pants','shirt','shoes','shorts','skirt','t-shirt']

In [25]:
pred = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [26]:
dict(zip(classes, pred[0]))

{'dress': -2.7052965,
 'hat': -4.6102924,
 'longsleeve': -0.9555743,
 'outwear': -1.890203,
 'pants': 8.090611,
 'shirt': -1.3633391,
 'shoes': -2.3920476,
 'shorts': 3.192402,
 'skirt': -3.025401,
 't-shirt': -2.0367362}

### 8.13 Summary

* We can use pre-trained models for general image classification
* Convolutional layers let us turn an image to a vector
* Dense layers use the vector to make the predictions
* Instead of training a model from scratch, we can use trasnfer learning and re-use already trained convolutional layers.
* First, train a small model (150x150) before training a big one (299x299)
* Learning rate - How fast the model trains. Fast learners aren't always best ones.
* We can save the best model using callbacks and checkpointing
* To avoid overfitting, use dropout and augmentation

### 8.14 Explore More

* Add more data, e.g Zalando,
* Albumentations - another way of generating augmentations
* Use PyTorch or MXNet instead of Tensorflow/Keras
* In addition to Xception, there are other architectures - try them

Other projects:
* Cats vs dogs
* Hotdog vs not hotddog
* Category of images